# Creating a simple forecasting model

In [1]:
library(reticulate)

## Installing Mosqlient

In [2]:
py_install("git+https://github.com/Mosqlimate-project/mosqlimate-client.git")

Using virtual environment '/home/fccoelho/Documentos/Projects_Software/sprint-template/.venv' ...


+ /home/fccoelho/Documentos/Projects_Software/sprint-template/.venv/bin/python -m pip install --upgrade --no-user 'git+https://github.com/Mosqlimate-project/mosqlimate-client.git'



In [3]:
mosq <- import("mosqlient")

In [4]:
mosq$version

[1] "1.2.0"